In [ ]:

from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.2MB 6.1MB/s 
     |████████████████████████████████| 870kB 24.4MB/s 
     |████████████████████████████████| 3.3MB 35.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=99e422504b420418a6f4cb4a2e1a89103e8162b88e71d18c39d24a9d021c33a6
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [ ]:
import sys

sys.path.insert(0,'drive/My Drive/sentiment/')


import torch
import numpy as np
import random
from numpy.random import seed
import pandas as pd
import pickle
import os
#import optuna
import tensorflow as tf

from keras.optimizers import schedules
from keras.models import Sequential,Model
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Input,concatenate, Bidirectional, SpatialDropout1D, GlobalAveragePooling1D
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from keras.regularizers import l1
from keras.losses import BinaryCrossentropy, CategoricalCrossentropy
from transformers import AutoTokenizer
from transformers import TFAutoModel,AutoModel
from transformers import TFXLMRobertaModel
from transformers import XLMRobertaTokenizer
from transformers import AdamWeightDecay, WarmUp
from transformers import BertConfig
import keras



In [ ]:
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: grpc://10.9.192.18:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.9.192.18:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
#import nlpaug.augmenter.char as nac


bert_multi = 'bert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(bert_multi)
print(tokenizer.vocab_size)
def bert_encoder(df,seq_len):
  X_ids = np.zeros((len(df),seq_len))
  X_mask = np.zeros((len(df),seq_len))
  for i,sentence in enumerate(df['text']):

      tokens = tokenizer.encode_plus(sentence,max_length=seq_len,truncation=True,padding='max_length',add_special_tokens=True,
                               return_token_type_ids=False,return_attention_mask=True,return_tensors='tf')
  
      X_ids[i,:],X_mask[i,:] = tokens['input_ids'], tokens['attention_mask']
  return X_ids, X_mask  
    


119547


In [ ]:
input_ids = Input(shape=(512,),name="input_ids",dtype=tf.int32)
mask = Input(shape=(512,),name="attention_mask",dtype=tf.int32)

config = BertConfig.from_pretrained(bert_multi,
                                    output_hidden_states=True)

model = TFAutoModel.from_pretrained(bert_multi,config=config)


Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
outputs = model(input_ids,mask)


In [ ]:
outputs

TFBaseModelOutputWithPooling([('last_hidden_state',
                               <KerasTensor: shape=(None, 512, 768) dtype=float32 (created by layer 'tf_bert_model_1')>),
                              ('pooler_output',
                               <KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'tf_bert_model_1')>),
                              ('hidden_states',
                               (<KerasTensor: shape=(None, 512, 768) dtype=float32 (created by layer 'tf_bert_model_1')>,
                                <KerasTensor: shape=(None, 512, 768) dtype=float32 (created by layer 'tf_bert_model_1')>,
                                <KerasTensor: shape=(None, 512, 768) dtype=float32 (created by layer 'tf_bert_model_1')>,
                                <KerasTensor: shape=(None, 512, 768) dtype=float32 (created by layer 'tf_bert_model_1')>,
                                <KerasTensor: shape=(None, 512, 768) dtype=float32 (created by layer 'tf_bert_model_1')>,
   

In [ ]:
outputs[2][9]

<KerasTensor: shape=(None, 512, 768) dtype=float32 (created by layer 'tf_bert_model_1')>

In [ ]:
'''vr = [var.name,var.shape for var in bert.trainable_variables]
vr'''

In [ ]:
seed(1)
tf.random.set_seed(seed=2)

seq_len = 512
EPOCHS = 4
BATCH_SIZE = 32
final_pred = []
train_data_size = 56000
steps_per_epoch = int(train_data_size / BATCH_SIZE)
num_train_steps = steps_per_epoch * EPOCHS
warmup_steps = num_train_steps * 0.1
AUTO = tf.data.AUTOTUNE




def train(fold_):
    
    print(f"Fold: {fold_}")
    print(" ")
    df = pd.read_csv('drive/My Drive/sentiment/inputs/stratified_5_folds.csv')
    
    # temporary dataframes for train and test
    train_df = df[df.kfold != fold_].reset_index(drop=True)

    val_df = df[df.kfold == fold_].reset_index(drop=True)
    # initialize CountVectorizer with NLTK's word_tokenize
    
    test_df = pd.read_csv('drive/MyDrive/sentiment/inputs/new_Test.csv')

    X_ids_train, X_mask_train = bert_encoder(train_df,seq_len)

    X_ids_val, X_mask_val = bert_encoder(val_df,seq_len)

    X_ids_test, X_mask_test = bert_encoder(test_df,seq_len)
    
    ytrain = train_df['label'].values
    yval = val_df['label'].values

    ytrain_enc = np_utils.to_categorical(ytrain)
    yval_enc = np_utils.to_categorical(yval)

    def example_to_features(input_ids,attention_masks,y):
        return {"input_ids": input_ids,"attention_mask": attention_masks},y

    def test_to_features(input_ids,attention_masks):
        return {"input_ids": input_ids,"attention_mask": attention_masks}    


    train_ds_one = tf.data.Dataset.from_tensor_slices((X_ids_train, X_mask_train, ytrain_enc)).map(example_to_features).cache().repeat().shuffle(1024).batch(BATCH_SIZE).prefetch(AUTO)
    val_ds = tf.data.Dataset.from_tensor_slices((X_ids_val, X_mask_val, yval_enc)).map(example_to_features).batch(BATCH_SIZE)
    test_ds = tf.data.Dataset.from_tensor_slices((X_ids_test, X_mask_test)).map(test_to_features).batch(BATCH_SIZE)

    with tpu_strategy.scope():
      input_ids = Input(shape=(seq_len,),name="input_ids",dtype=tf.int32)
      mask = Input(shape=(seq_len,),name="attention_mask",dtype=tf.int32)
      inp_dict = {"input_ids": input_ids,"attention_mask": mask}
      config = BertConfig.from_pretrained(bert_multi,
                                   output_hidden_states=True)
      
      bert = TFAutoModel.from_pretrained(bert_multi,config=config)
      #bert.trainable = False
      outputs = bert(input_ids)
      x = Conv1D(128, kernel_size = 7, padding = "valid", kernel_initializer = "he_uniform")(outputs[0])
      x = GlobalAveragePooling1D()(x)
      y = Dense(3,activation='softmax',name='y')(x)
     

      model = Model(inputs=inp_dict, outputs=y)      
      model.compile(loss=CategoricalCrossentropy(),steps_per_execution = 50,
                    optimizer=tf.keras.optimizers.Adam(lr=2e-5)
                    ,metrics=['accuracy'])
      model.summary()

    earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='min')
    save_best = ModelCheckpoint(f"drive/My Drive/sentiment/models/modeln{fold_}.h5", save_best_only=True,save_weights_only=True, monitor='val_loss', mode='min')
    rl = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, min_delta=1e-4, mode='min')
    #class_weight = {2: 0.09, 0: 0.12, 1: 1.42}
    model.fit(train_ds_one, epochs=EPOCHS, 
          verbose=1, validation_data=val_ds,steps_per_epoch=steps_per_epoch, callbacks=[earlystop,save_best,rl])

    predictions = model.predict([X_ids_test, X_mask_test])
    final_pred.append(predictions)
    return final_pred







In [ ]:
for i in range(5):
  pr = train(fold_=i)

Fold: 0
 


Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 512)]        0                                            
__________________________________________________________________________________________________
tf_bert_model_1 (TFBertModel)   TFBaseModelOutputWit 177853440   input_ids[0][0]                  
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 506, 128)     688256      tf_bert_model_1[0][13]           
__________________________________________________________________________________________________
global_average_pooling1d_1 (Glo (None, 128)          0           conv1d_1[0][0]                   
____________________________________________________________________________________________

1750/1750 [==============================] - ETA: 0s - loss: 0.6767 - accuracy: 0.6865WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


1750/1750 [==============================] - 328s 187ms/step - loss: 0.6766 - accuracy: 0.6866 - val_loss: 0.4817 - val_accuracy: 0.7960
Epoch 2/4
1750/1750 [==============================] - 222s 127ms/step - loss: 0.4614 - accuracy: 0.8051 - val_loss: 0.4580 - val_accuracy: 0.8059
Epoch 3/4
1750/1750 [==============================] - 222s 127ms/step - loss: 0.3714 - accuracy: 0.8492 - val_loss: 0.4733 - val_accuracy: 0.8046
Epoch 4/4
1750/1750 [==============================] - 222s 127ms/step - loss: 0.2867 - accuracy: 0.8879 - val_loss: 0.5570 - val_accuracy: 0.8117

Epoch 00004: ReduceLROnPlateau reducing learning rate to 1.9999999494757505e-06.


...

Fold: 4
 


Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 512)]        0                                            
__________________________________________________________________________________________________
tf_bert_model_5 (TFBertModel)   TFBaseModelOutputWit 177853440   input_ids[0][0]                  
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 506, 128)     688256      tf_bert_model_5[0][13]           
__________________________________________________________________________________________________
global_average_pooling1d_5 (Glo (None, 128)          0           conv1d_5[0][0]                   
____________________________________________________________________________________________

1750/1750 [==============================] - ETA: 0s - loss: 0.7016 - accuracy: 0.6711WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


1750/1750 [==============================] - 328s 187ms/step - loss: 0.7015 - accuracy: 0.6711 - val_loss: 0.5027 - val_accuracy: 0.7895
Epoch 2/4
1750/1750 [==============================] - 223s 128ms/step - loss: 0.4752 - accuracy: 0.7991 - val_loss: 0.4698 - val_accuracy: 0.8014
Epoch 3/4
1750/1750 [==============================] - 223s 127ms/step - loss: 0.3891 - accuracy: 0.8387 - val_loss: 0.4740 - val_accuracy: 0.8145
Epoch 4/4
1750/1750 [==============================] - 222s 127ms/step - loss: 0.3120 - accuracy: 0.8758 - val_loss: 0.5508 - val_accuracy: 0.8162

Epoch 00004: ReduceLROnPlateau reducing learning rate to 1.9999999494757505e-06.


In [ ]:
pred_0 = np.concatenate(pr,axis=0)
print(pred_0)
print(pred_0.shape)
#print(np.argmax(pred_0,axis=1))
n_pred = pred_0.reshape(5,30000,3)
avg = np.mean(n_pred,axis=0)
#print(avg.shape)
y_classes = avg.argmax(axis=-1)
print(y_classes)

In [ ]:
target = {-1: 0, 0: 1, 1: 2}
idx_to_class = {val:key for key ,val in target.items()}
print(idx_to_class)

(150000, 3)
[0 0 2 ... 2 0 2]


In [ ]:
test_df = pd.read_csv('drive/MyDrive/sentiment/inputs/Test.csv')
sub = pd.DataFrame()
sub["ID"] = test_df["ID"]
sub["label"] = y_classes
sub['label'].replace(idx_to_class, inplace=True)
sub.head()
sub.to_csv('drive/MyDrive/sentiment/inputs/sub.csv', index=False)

{0: -1, 1: 0, 2: 1}
